In [ ]:
Author : Saurabh Manga
Run this only on 2 L4 GPUs otherwise the Kernel will die off for no reasason, also can be run on sinhle A100 / H100

In [ ]:
# git@github.com:analyticsrepo01/flux_text2image.git

In [ ]:
# !pip install git+https://github.com/huggingface/diffusers.git

!pip install torch	torchvision	safetensors	git+https://github.com/huggingface/diffusers.git	transformers	lycoris-lora==1.8.3	flatten_json	pyyaml	oyaml	tensorboard	kornia	invisible-watermark	einops	accelerate	toml	albumentations	pydantic	omegaconf	k-diffusion	open_clip_torch	timm	prodigyopt	controlnet_aux==0.0.7	python-dotenv	bitsandbytes	hf_transfer	lpips	pytorch_fid	optimum-quanto	sentencepiece	huggingface_hub	peft

!pip install dotenv wrapper

In [10]:
from huggingface_hub import login
import os
from env import load_dotenv
import json

load_dotenv('.env')
HUGGING_FACE_HUB_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN")

print("HUGGING_FACE_HUB_TOKEN", HUGGING_FACE_HUB_TOKEN)

try:
    login() 
    print("Login successful!")
except Exception as e:
    print(f"Login failed: {e}") 

ModuleNotFoundError: No module named 'wrapper'

In [6]:
import torch
from diffusers import FluxPipeline
model_name = "black-forest-labs/FLUX.1-schnell"
              
pipe = FluxPipeline.from_pretrained(model_name, torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power



prompt = "A cat holding a sign that says hello world"
image = pipe(
    prompt,
    height=1024,
    width=1024,
    guidance_scale=3.5,
    output_type="pil",
    num_inference_steps=50,
    max_sequence_length=512,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image.save("flux-dev.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: load_checkpoint_and_dispatch() got an unexpected keyword argument 'strict'

In [7]:
from transformers import AutoModelForCausalLM # or appropriate model class

# Load without 'strict' 
model = AutoModelForCausalLM.from_pretrained(model_name)

# If 'strict' is needed in a newer version, update Transformers and use it as intended

OSError: black-forest-labs/FLUX.1-schnell does not appear to have a file named config.json. Checkout 'https://huggingface.co/black-forest-labs/FLUX.1-schnell/tree/main' for available files.

In [12]:

!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd ./TotoroUI

fatal: could not create leading directories of '/content/TotoroUI': Permission denied
[Errno 2] No such file or directory: './TotoroUI'
/home/jupyter/GenAI6/flux_text2image


In [ ]:


# !pip install -q torchsde einops diffusers accelerate xformers==0.0.27
# !apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/flux1-schnell.sft -d /content/TotoroUI/models/unet -o flux1-schnell.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-schnell.sft", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2